- main runs = VD
- reverse runs = DV
- TODO: LIVE MAPPING VISUAL BLOCK ANALYSIS ON WHOLE BRAIN
- Correct slice thickness (alex said some stuff) 
- Correct position of the coil to see both eyes
- Try LCD on Friday, if not done already
- Design stimulus on LCD
- Container environment
    - FSL, Freesurfer, AFNI, ANTS[latest:2.4.3], c3d, workbench
    - python
        - jupyterlab, numpy, nibabel, nilearn, scikit-learn, pandas, matplotlib, astropy, sklearn, 
        - to add: nipype, niworkflows, scipy
- 9.4T description info:
    - task info
    - distortion direction
    - resolution 

In [ ]:
from pathlib import Path
test_dir='/data/mouse_data/bids/derivatives/notebook_analysis/TEST'
!rm -rf {test_dir}
Path(test_dir).mkdir(parents=True,exist_ok=True)

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import sys

sys.path.append('/opt/app')

from fastfmri_toolbox.modelling.design_matrix import DesignMatrix
from fastfmri_toolbox.modelling.first_level_analysis import FirstLevelAnalysis

import nibabel as nib

In [ ]:
def plot_slices(nifti_path,overlay=None,title='',z_threshold = 2.,_cmap='magma', underlay_vmax=30000,overlay_type='scatter'):
    import nibabel as nib
    import numpy as np
    import matplotlib.pyplot as plt
    import matplotlib.cm as cm

    nii = nib.load(nifti_path)
    data = nii.get_fdata()
    
    for z in range(data.shape[2]):
        if z % 5 == 0:
            fig, axs = plt.subplots(nrows=1, ncols=5, figsize=(12, 3))
    

        im1 = axs[z % 5].imshow(data[:, :, z], cmap='gray',vmax=underlay_vmax)
        if overlay is not None:
            overlay_data = nib.load(overlay).get_fdata()
            cmap = np.zeros_like(overlay_data)
            overlay_coords = np.where(overlay_data>z_threshold)
            cmap[overlay_coords] = overlay_data[overlay_coords]
            
            # overlaid imshow
            if overlay_type == 'imshow':
                im2 = axs[z % 5].imshow(cmap[:, :, z], cmap=_cmap, alpha=.5)
            
            # Scatter plot
            if overlay_type == 'scatter':
                _xs, _ys, _zs = overlay_coords[1], overlay_coords[0], overlay_coords[2]
                x_coords_z_slice, y_coords_z_slice, intensities = [], [], []
                for _x, _y, _z in zip(_xs,_ys,_zs):
                    if _z == z:
                        x_coords_z_slice.append(_x-.25)
                        y_coords_z_slice.append(_y-.25)
                        intensities.append(overlay_data[_y,_x,_z])
                scatter = axs[z % 5].scatter(
                    x_coords_z_slice,
                    y_coords_z_slice,
                    c=intensities,
                    s=3.,
                    cmap=_cmap,
                    vmin=0,
                    vmax=overlay_data.max(),
                )
                # Create a ScalarMappable object for the colorbar
                try:
                    sm = cm.ScalarMappable(
                        cmap=_cmap, 
                        norm=plt.Normalize(
                            vmin=0,
                            vmax=overlay_data.max(),
                        )
                    )
                    sm._A = []  # Needed for matplotlib v3.3 and above
                    # Add a colorbar to the plot
                    cbar = fig.colorbar(sm, ax=axs[z%5])
                except:
                    pass

        axs[z % 5].axis('off')
        try:
            fig.suptitle(f"{title}\nMax Z-score: {overlay_data.max():.2f}")
        except:
            fig.suptitle(f"{title}")

        if (z + 1) % 5 == 0 or z == data.shape[2] - 1:
            plt.tight_layout()
            plt.show()

#### Get data and create masks

In [ ]:
frequency_encoding_runs = !ls /data/mouse_data/bids/sub-06393073M/ses-Pilot01/func/*VisualFT*.nii.gz

mm = .5
sigma = mm / 2.355
masks, tmeans, smooth_bolds = [], [], []
for p in frequency_encoding_runs:
    
    p = Path(p)
    basename = p.stem.split('.nii')[0]
    
    smooth_name = f"{test_dir}/{basename.replace('bold','smoothedbold')}.nii.gz"
    !fslmaths {p} -s {sigma} {smooth_name}
    assert Path(smooth_name).exists(),f"{smooth_name} does not exist."
    
    tmean_name = f"{test_dir}/{basename.replace('bold','tmean')}.nii.gz"
    !fslmaths {p} -Tmean {tmean_name}
    assert Path(tmean_name).exists(),f"{tmean_name} does not exist."
    
    mask_name = f"{test_dir}/{basename.replace('bold','mask')}.nii.gz"
    !fslmaths {p} -Tmean -bin {mask_name}
    assert Path(mask_name).exists(),f"{mask_name} does not exist."
    
    smooth_bolds.append(smooth_name)
    tmeans.append(tmean_name)
    masks.append(mask_name)


In [ ]:
smooth_bolds

In [ ]:
avg = '/data/mouse_data/bids/derivatives/notebook_analysis/TEST/sub-06393073M_ses-Pilot01_task-TEST02VisualFT_dir-AP_run-N3AVERAGE_smoothedbold.nii.gz'
!fslmaths \
    {smooth_bolds[4]} \
    -add {smooth_bolds[5]} \
    -add {smooth_bolds[6]} \
    -div 3 \
    {avg}

smooth_bolds.append(avg)
tmeans.append(tmean_name)
masks.append(mask_name)

#### Test plotting

In [ ]:
plot_slices(tmeans[6],title='Temporal mean')

#### Get z-score

In [ ]:
%%time

search_frequencies = [.2]

run_one = False
for ix, (run, mask) in enumerate(zip(smooth_bolds,masks)):
    # Get run info to calculate time window, a beginning time of 40s is arbitrarily chosen
    img = nib.load(run)
    tr = img.header.get_zooms()[-1]
    n_vols = img.shape[-1]
    
    time_window = (
        50, 
        int(tr*n_vols)
    )
    dm = DesignMatrix(time_window, search_frequencies, bold_path=run)
    design_matrix = dm.build_design_matrix()

    fla = FirstLevelAnalysis(
        derivatives_dir = test_dir,
        bold_path = run,
        mask_path = mask,
        design_matrix = design_matrix,
        time_window = time_window,
        search_frequencies = search_frequencies
    )
    fla.run_frequency_glm(save_windowed_bold = True, save_predicted = True, save_residual = True)
    
    if ix == 0 and run_one:
        break

In [ ]:
!tree {test_dir}

In [ ]:
z_scores = !ls {test_dir}/sub-06393073M/ses-Pilot01/task-*/*/GLM/frequency-0.2_z_score.nii.gz
z_scores

In [ ]:
for ix, (run, tmean,mask,z_score) in enumerate(zip(smooth_bolds,tmeans,masks,z_scores)):
    plot_slices(tmean,z_score,title=Path(run).stem.split('.nii')[0],z_threshold=2.,_cmap='magma',overlay_type='scatter')

In [ ]:
"""_dm = dm.build_design_matrix()
_dm.iloc[:20]"""

In [ ]:
!ls /data/mouse_data/bids